In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('parlimen-24k-chunk_processed/**/*/*.json', recursive = True)

len(files)

3399

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [4]:
!rm -rf parliament-chunk
!mkdir parliament-chunk

In [5]:
def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = -10, minimum_length = 5.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            scores = [a for a in align if a['score'] <= -15]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    try:
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('parliament-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [ ]:
d = loop((files[-100:], 0))

In [ ]:
len(d)

In [ ]:
d[-1]

In [ ]:
import IPython.display as ipd
ipd.Audio(d[-1]['audio'])

In [6]:
data = mp.multiprocessing(files, loop, cores = 15)

100%|███████████████████████████████████████████████████████████████████████████████████████| 226/226 [35:56<00:00,  9.54s/it]


In [8]:
len(data)

191545

In [9]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|████████████████████████████████████████████████████████████████████████████| 191545/191545 [00:00<00:00, 2483678.47it/s]


defaultdict(int, {'parliament-chunk/parlimen-24k-chunk': 191545})

In [10]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,audio,transcription
0,parliament-chunk/parlimen-24k-chunk_processed_...,dewan rakyat dengan menyebut nama Allah Yang M...
1,parliament-chunk/parlimen-24k-chunk_processed_...,"yang bersifat maha hikmat, limpah keadilan, ya..."
2,parliament-chunk/parlimen-24k-chunk_processed_...,permesyuaratan bagi mentadbir maslahat masyara...
3,parliament-chunk/parlimen-24k-chunk_processed_...,saya ingin menarik perhatian pihak lembaga pem...
4,parliament-chunk/parlimen-24k-chunk_processed_...,satu sistem pendidikan yang mempunyai aliran y...


In [11]:
df.to_parquet('verify-text-chunk-parliament.parquet')

In [12]:
!du -hs parliament-chunk

11G	parliament-chunk


In [14]:
!zip -q -r -s 10000m parliament-chunk.zip parliament-chunk

In [15]:
!ls parliament-chunk.z*

parliament-chunk.zip


In [13]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-parliament.parquet",
    path_in_repo="data/parliament_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


verify-text-chunk-parliament.parquet:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/49022a2f3c2d58a303015445d0772fac573c9a27', commit_message='Upload data/parliament_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='49022a2f3c2d58a303015445d0772fac573c9a27', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)

In [16]:
api.upload_file(
    path_or_fileobj="parliament-chunk.zip",
    path_in_repo="data/parliament-chunk.zip",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)

parliament-chunk.zip:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/2aa6e9ada6d9cb2630b41cea23cb459efed6545a', commit_message='Upload data/parliament-chunk.zip with huggingface_hub', commit_description='', oid='2aa6e9ada6d9cb2630b41cea23cb459efed6545a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)